In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# data loading

import pickle

data_path = '../data/2016-10-20-news-bow.pkl'
SAVE_RESULT = False

with open(data_path, 'rb') as f:
    params = pickle.load(f)
    x = params['x']
    index2word = params['index2word']
    word2index = params['word2index']

print(x.shape)
print(word2index['아이오아이'])

(30091, 9774)
5537


In [3]:
# train spherical k-means model

from soyclustering import SphericalKMeans
from sklearn.preprocessing import normalize

kmeans = SphericalKMeans(
    n_clusters = 500,
    init = 'similar_cut',
    verbose = True
)

x = normalize(x, norm='l2')
labels = kmeans.fit_predict(x)

initialization_time=1.218108 sec, sparsity=0.00796
n_iter=1, changed=29969, inertia=15323.440, iter_time=4.435 sec, sparsity=0.116
n_iter=2, changed=5062, inertia=11127.620, iter_time=4.466 sec, sparsity=0.108
n_iter=3, changed=2179, inertia=10675.314, iter_time=4.463 sec, sparsity=0.105
n_iter=4, changed=1040, inertia=10491.637, iter_time=4.449 sec, sparsity=0.103
n_iter=5, changed=487, inertia=10423.503, iter_time=4.437 sec, sparsity=0.103
n_iter=6, changed=297, inertia=10392.490, iter_time=4.483 sec, sparsity=0.102
n_iter=7, changed=178, inertia=10373.646, iter_time=4.442 sec, sparsity=0.102
n_iter=8, changed=119, inertia=10362.625, iter_time=4.449 sec, sparsity=0.102
n_iter=9, changed=78, inertia=10355.905, iter_time=4.438 sec, sparsity=0.102
n_iter=10, changed=80, inertia=10350.703, iter_time=4.452 sec, sparsity=0.102


In [4]:
# cluster labeling

from soyclustering import proportion_keywords

centers = kmeans.cluster_centers_
keywords = proportion_keywords(
    centers,
    labels,
    index2word = index2word
)

CPU times: user 508 ms, sys: 4 ms, total: 512 ms
Wall time: 514 ms


In [5]:
centers.shape

(500, 9774)

In [5]:
from collections import Counter

cluster_size = Counter(labels)
selected_clusters = {
    idx for idx in range(centers.shape[0])
    if 50 <= cluster_size.get(idx) <= 100
}

print('number of selected clusters = {}'.format(len(selected_clusters)))

number of selected clusters = 117


In [6]:
if SAVE_RESULT:
    with open('./2016-10-20-news-kmeans500.pkl', 'wb') as f:
        params = {
            'centers': centers,
            'labels': labels,
            'index2word': index2word
        }
        pickle.dump(params, f)

In [7]:
topk = 50

def pretty_label_print(labels, words_in_a_line=10):
    for i, word in enumerate(labels):
        print('{}, '.format(word), end='')
        if (i + 1) % words_in_a_line == 0:
            print()
    print()

for cluster_idx in list(selected_clusters)[:10]:
    cluster_labels = [
        word for word, score in sorted(
            keywords[cluster_idx], key=lambda x:-x[1])[:topk]
    ]

    print('\n# cluster = {}'.format(cluster_idx))
    pretty_label_print(cluster_labels)


# cluster = 0
당사국, 해상, 평택, 5차례, 공사비, 남쪽, 몰래, 총회, 등재, 370, 
항만, 당진, 적법, 수산, 올리기, 깊게, 상업, 얻게, 낭비, 면적, 
배출, 국토, 발급, 해도, 식별, 해양수산부, 저해, 본토, 수산물, 연휴, 
투기, 협정, 의무화, 해수부, 아리아, 선장, 국제적, 급감, 256, 유럽연합, 
구축함, 절감, 작성, 해경, 2천, 부실, 간부, 삭제, 동쪽, 협약, 


# cluster = 3
박수진, 남성미, 화보, 몽환적, 시크, 매거진, 오연서, 패셔니스타, 입술, 11월호, 
메이크업, 빛이, 촉촉, 탄성, 예민, 소신, 최성, 빠지게, 진한, 반짝, 
아낌없, 보석, 섹시함, 따스, 물씬, 오렌지, 하이, 환절기, 수분, 이종석, 
원피스, 소화, 몸매, 데이즈, 돋보, 정수, 에이핑크, 김남주, 실루엣, 정체성, 
훌륭, 화장, 러블리, 입어, 햇살, 과감, 정은지, 진솔, 발간, 스타일링, 


# cluster = 17
논현동, 런칭, 양윤영, 강소영, 김진솔, 박희, 미스코리아, 서인영, 손예진, 라움, 
론칭, 화장품, 미샤, 고아라, 역삼동, 이탈, 시리즈, 이지, 기념, 화사, 
강남구, 뉴스1스타, 시계, 신제품, 구두, 우아, 스포츠조선, 취하고, 포즈, 행사, 
헤럴드, 브랜드, 가수, 어마어마, 자태, 자신감, 원피스, 모델, 출시, 하트, 
스타, 발산, 다국적, 참석, 러블리, 배우, 무브먼트, 곡선, 눈빛, 케이스, 


# cluster = 18
총기사고, 윤동, 벌이기, 부엌, 묵념, 모니터, 번동파출소, 법무부, 동료들, 추모, 
오패산, 공조, 터널, 1세대, 내구성, 경관, 전자발찌, 순직, 문제점, 김주, 
강북구, 강북경찰서, 성모씨, 경감, 용의자, 살해, 직전, 리본, 진화, 성병대, 
잠시, 총격, 경찰관, 헬멧, 2008년, 달고, 경찰서, 김창, 숨지게, 출동, 
방탄조끼, 김동, 폭죽, 하기, 붙어, 서바이벌, 사제, 관리, 검거, 신발, 


# 